In [ ]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/tehdog/data/dev/2019/pic2kcal-cv-praktikum:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [94]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece
!pip install tqdm

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 51kB 2.0MB/s ta 0:00:011
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0630 17:55:02.529980 140551823103744 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [3]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from math import ceil

data_dir = Path("data")

In [4]:
# jq '.[]' fddb_data_v3.json > fddb_data_v3.jsonl
# jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' fddb_data_v3.jsonl | jq -s > fddb_data_v3_withimg.json
with open(str(data_dir / "fddb_data_v3_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [194]:
def normalize_ingredient(ing: str):
    ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing)
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

In [201]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
len(in_names)

21444

In [202]:
len(out_names)

122977

In [203]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [204]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [205]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [206]:
def get_match(search: np.array, out_vecs):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(10, it, key=itemgetter(1))
    return res_list

In [207]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [208]:
out_vecs[0][1].shape

(512,)

In [209]:
in_names[0], out_names[0]

('Currypulver nach Wahl', 'Get Fruity Bar, Marvellous Mango')

In [210]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [212]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

Soja -Granulat nach Packungsanweisung vorbereitet -> ('Soja Würfel rein pflanzlich', 0.69715405)
Sojamilch -> ('Sojamilch', 0.9999999)
Semmelbrösel bzw. Paniermehl nach Belieben -> ('Semmelbrösel, aus Weißbrot', 0.8380441)
Suppengemüse -> ('Suppengemüse', 1.0)
Gemüse (z.B. Kartoffeln -> ('Gemüseschiffchen, Kartoffel, Gemüse', 0.8411544)
Blüten vom Kürbis -> ('Kürbis, frisch', 0.83566153)
Koriander + -> ('Bohnensalat +', 0.7269006)
Zitronenschale und -saft -> ('Zitronensaft', 0.8968452)
Bohnen  abgetropt -> ('Bohneneintopf', 0.90160286)
Garnelen TK -> ('Garnelen Edeka ', 0.7927007)
Wild - Schweinbraten -> ('Schweine - Filet', 0.836059)
Fleischreste vom Brathähnchen -> ('Hähnchenhackfleisch ', 0.8774542)
Calazoon -> ('Arginin', 0.82420176)
Paprikamark aus der Tube -> ('Paprikamark', 0.830608)
Früchte zur Garnitur -> ('Früchtemüsli', 0.7896198)
Petersilie und Dill -> ('Petersilie, frisch', 0.8730626)
Erdmandelmehl -> ('Grünkernmehl', 0.89057696)
Pinienkerne und Cashewnüsse -> ('Cashewnüss

In [215]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

[('Kartoffeln', 0.99999994),
 ('Kartoffelstock', 0.8959534),
 ('Kartoffelstock', 0.8959534),
 ('Kartoffeln roh', 0.8905238),
 ('Kartoffeln gebacken', 0.8904908),
 ('Kartoffelscheiben', 0.8901619),
 ('Kartoffelscheiben', 0.8901619),
 ('Kartoffeln, ganz', 0.88885343),
 ('Kartoffelnudeln', 0.8792578),
 ('Gekochte Kartoffeln', 0.8718474)]

In [56]:
from multiprocessing import Pool
def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(8) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = match

In [64]:
def jsonable(e):
    return [(a,float(b)) for a, b in e]
all_ings_orig = {ing: jsonable(all_ings[normalize_ingredient(ing)]) for ing in _in_names}

In [65]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)

In [66]:
#


In [140]:
[ov for ov in out_vecs if ov[0] == 'Butter']

[('Butter',
  array([-0.00069197, -0.00708851,  0.07452937,  0.00758318,  0.05222673,
          0.03806081,  0.05294695,  0.03213596,  0.02329489,  0.05352827,
         -0.05112645,  0.09030724,  0.04542807, -0.0291456 ,  0.06813187,
          0.00724201, -0.03245255, -0.00128621, -0.03777418,  0.05070751,
         -0.00641798, -0.00933374,  0.05987255, -0.04853407,  0.02772648,
          0.03996199,  0.00149899, -0.03556515, -0.03610078, -0.01356451,
         -0.02494853,  0.03070905,  0.01266939, -0.01730152,  0.06919695,
         -0.05880246, -0.01854649,  0.01752839, -0.00583083,  0.0338078 ,
         -0.05956725,  0.05600455,  0.01535737, -0.01911689,  0.04840055,
          0.05798056, -0.02095032,  0.02836767, -0.00945359, -0.04201555,
         -0.05017705, -0.02037028, -0.04847089,  0.05416684,  0.02418666,
          0.07717222, -0.04641171,  0.01365955, -0.06059999, -0.03713608,
         -0.0222505 , -0.01606266,  0.0571235 , -0.05619613, -0.00114614,
          0.00769384,  0.0

In [152]:
re.match("^Nährwerte für 100 (\w+)$", "Nährwerte für 100 g").groups()

('g',)

In [162]:
from collections import defaultdict
fddb_entries = defaultdict(list)
rem = re.compile(r"^(\d+ )?(.*) \((\d+(?:,\d+)?) (g|ml)\)$")
for e in fddb:
    fddb_entries[e["name"]].append(e)
    sanity = list(e["Standard Nährwerte"].keys())
    snmatch = re.match("^Nährwerte für 100 (\w+)$", sanity[0])
    if snmatch is not None:
        unn = snmatch.groups()[0]
        e["parsed"] = {"unit": unn, "for_100": e["Standard Nährwerte"]["Nährwerte für 100 {}".format(unn)]}
        
    else:
        print("SANITY ERROR:", e["Id"], sanity, unit)
        continue
    amts = e["parsed"]["amounts"] = []
    for k in e["Spezifische Nährwerte"].keys():
        if k == "100 g (100 g)" or k == "100 g (100 ml)" or k == "100 ml (100 ml)":
            continue
        txtamount, txt, amount, unit = rem.match(k).groups()
        amount = float(amount.replace(",", "."))
        txtamount = float(txtamount.replace(",", ".")) if txtamount else 1.0
        amts.append((txt, k, amount/txtamount))
        
        if unn != unit:
            raise Exception("noooo")
        #print(txt, amount/txtamount, unit)

SANITY ERROR: https://fddb.info/db/de/lebensmittel/selbst_gemacht_100_kalorien_dummy/index.html ['Nährwerte deines Gerichts kennst und in dein Ernährungstagebuch eintragen möchtest.\n31 Bewertungen\nSchreibe eine Bewertung\nHersteller: Selbst gemacht\nProduktgruppe: Sonstiges\nDatenquelle: Schätzung. Die Produktdaten wurden am 18.05.2006 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 08.11.2018. '] g
SANITY ERROR: https://fddb.info/db/de/lebensmittel/weider_weider_yippie_riegel_cookies_netto_naehrwerte/index.html ['Nährwerte) von MarieCarlotta | Hochgeladen von: MarieCarlotta (Problem melden) Details\n\nFoto hochladen\nNoch keine Beschreibung für dieses Produkt.\nAngaben noch nicht bestätigt. Melde einen Fehler oder korrigiere die Angaben.\nSchreibe eine Bewertung\nHersteller: Weider\nProduktgruppe: Nahrungsergänzung\nDatenquelle: Extern. Die Produktdaten wurden am 09.11.2017 von einem Fddb Nutzer erhoben. Hinweise zu den Produktdaten. Aktualisiert: 04.07.20

In [264]:
# get recipe meta
with open("data/recipes/processed_data.json", encoding="utf-8") as f:
    recipes = json.load(f)

In [265]:
# get amount map from parse-amounts
with open("data/recipes/parsed_amounts.json") as f:
    parsed_amounts = json.load(f)

In [266]:
# only recipes with pictures
recipes = [recipe for recipe in recipes if len(recipe['picture_files']) > 0]

In [274]:
from operator import itemgetter
# ugly af code
debug=False
for recipe in random.sample(recipes, 10):
    print("----------------------------")
    print("" + recipe["title"])
    for ingredient in recipe["ingredients"]:
        if "amount" in ingredient:
            print((ingredient["amount"].rjust(15) + " " + ingredient["ingredient"]).ljust(55), end='')
            print(" = ", end='')
            _z = parsed_amounts[ingredient["amount"]]
            if _z is None:
                print(ingredient["amount"])
            amount_count, amount_type = _z
            
            if amount_count is None:
                print(" [ignored]")
                continue
            if amount_type is None:
                amount_type = "Stück"
            #print("[" + str(amount_count) + "*" + (amount_type)+ "] of ", end="")
            _ing_matches = all_ings_orig[ingredient["ingredient"]]
            ing_matches = [e for e in _ing_matches if e[1] > 0.85]
            if len(ing_matches) == 0:
                # not so great match but eh
                ing_matches = [_ing_matches[0]]
            possible_entries = {entry["Id"]: (ing_match[1], entry) for ing_match in ing_matches for entry in fddb_entries[ing_match[0]]}
            possible_entries = sorted(list(possible_entries.values()), key=lambda e: -e[0])

            debug and print("matches", ing_matches)
            debug and print("finding unit ", amount_type)
            match = None
            for accuracy, e in possible_entries:
                if "parsed" not in e:
                    continue
                debug and print("considering", e["name"], e["Id"])
                have_unit = [amount_type]
                want_unit = e["parsed"]["unit"]
                
                if want_unit in have_unit:
                    debug and print("got direct match!", amount_count, have_unit, amount_count, want_unit)
                    match = (e, accuracy, amount_count, want_unit, amount_count, want_unit)
                    break
                for amount_name, amount_source, to_norm in e["parsed"]["amounts"]:
                    debug and print(e["name"], "amount", amount_source)
                    if amount_name in have_unit:
                        debug and print("got match unit!", amount_source)
                        match = (e, accuracy, amount_count, have_unit[0], amount_count*to_norm, want_unit)
                        break
                if match is not None:
                    break
            if not match:
                print("[nomatch]")
            else:
                fddb_entry, accuracy, count_weird, unit_weird, count_norm, unit_norm = match
                if unit_weird == unit_norm:
                    unittxt = "{}{}".format(count_norm, unit_norm)
                else:
                    unittxt = "{} {} = {}{}".format(count_weird, unit_weird, count_norm, unit_norm)
                values = e["parsed"]["for_100"]
                value_kcal_per_100 = float(values["Kalorien"]["Menge"].replace(",", "."))
                value_kcal = value_kcal_per_100 / 100 * count_norm
                print("({:.0f}%) [{}] of {} = {:.0f} kcal".format(accuracy * 100, unittxt, fddb_entry["name"], value_kcal))
        else:
            print("->" + ingredient["subtitle"])

----------------------------
Apfelnudeln à la Oma Langer
          500 g Mehl                                    = (100%) [500g] of Mehl = 1740 kcal
       1 Würfel Hefe, frische                           = [nomatch]
         200 ml Milch, warme                            = (100%) l] of Milch = 128 kcal
          100 g Zucker                                  = (100%) [100g] of Zucker = 405 kcal
           90 g Butter, oder Margarine                  = (100%) [90g] of Butter = 530 kcal
     1 Prise(n) Salz                                    = (100%) [1 Prise = 1.0g] of Salz = 0 kcal
        6 große Äpfel                                   = [nomatch]
              1 Ei(er)                                  = [nomatch]
         1 Pck. Vanillezucker                           = (100%) [1 Packung = 8.0g] of Vanillezucker = 32 kcal
         1 Msp. Zimtpulver                              = [nomatch]
½ kl. Flasche/n Aroma, (Butter-Vanillearoma)            = [nomatch]
          etwas Krokant, bei